In [1]:
!pip install spacy rouge-score datasets tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=557c46ea0f0750fd323b07738a5239b9e0674adc84dd44f2841cf829912f2bad
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are in

In [14]:
!pip install scikit-learn

In [15]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
import spacy
from heapq import nlargest
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS

In [ ]:
from rouge_score import rouge_scorer
from tqdm import tqdm

In [94]:
from datasets import load_dataset

dataset = load_dataset("EdinburghNLP/xsum", split="test")


ds = load_dataset("ccdv/govreport-summarization", split="test")


In [5]:
print(dataset[0])

{'document': 'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the

In [95]:
nlp = spacy.load("en_core_web_lg")

def summarize_text(text, compression_ratio=0.3):
    try:
        doc = nlp(text)
        stopwords = list(STOP_WORDS)
        word_frequencies = {}

        for word in doc:
            if word.text.lower() not in stopwords and word.text.lower() not in punctuation:
                word_frequencies[word.text.lower()] = word_frequencies.get(word.text.lower(), 0) + 1

        max_frequency = max(word_frequencies.values(), default=1)
        word_frequencies = {word: freq / max_frequency for word, freq in word_frequencies.items()}

        sentence_scores = {}
        for sentence in doc.sents:
            for word in sentence:
                if word.text.lower() in word_frequencies:
                    sentence_scores[sentence] = sentence_scores.get(sentence, 0) + word_frequencies[word.text.lower()]

        select_length = int(len(list(doc.sents)) * compression_ratio)
        summarized_sentences = nlargest(select_length, sentence_scores, key=sentence_scores.get)
        summary = ' '.join([sentence.text for sentence in summarized_sentences])
        return summary
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return text


In [97]:
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

num_samples = 100
documents = dataset["document"][:num_samples]
reference_summaries = dataset["summary"][:num_samples]

rouge_scores = {"rouge1": {"precision": [], "recall": [], "f1": []},
                "rouge2": {"precision": [], "recall": [], "f1": []},
                "rougeL": {"precision": [], "recall": [], "f1": []}}

for doc, ref_summary in tqdm(zip(documents, reference_summaries), total=num_samples):
    predicted_summary = summarize_text(doc)
    scores = scorer.score(predicted_summary, ref_summary)

    for key in rouge_scores:
        rouge_scores[key]["precision"].append(scores[key].precision)
        rouge_scores[key]["recall"].append(scores[key].recall)
        rouge_scores[key]["f1"].append(scores[key].fmeasure)

avg_rouge = {key: {metric: (sum(values) / num_samples) * 100
                    for metric, values in rouge_scores[key].items()}
             for key in rouge_scores}

print("ROUGE Scores (Precision, Recall, F1-score):")
for metric, values in avg_rouge.items():
    print(f"{metric.upper()}: Precision={values['precision']:.4f}, Recall={values['recall']:.4f}, F1={values['f1']:.4f}")


100%|██████████| 100/100 [00:06<00:00, 14.84it/s]

ROUGE Scores (Precision, Recall, F1-score):
ROUGE1: Precision=45.8608, Recall=8.6142, F1=13.2099
ROUGE2: Precision=11.0209, Recall=1.5934, F1=2.6061
ROUGEL: Precision=31.3024, Recall=5.8031, F1=8.8498
